In [9]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import OneCycleLR

import matplotlib.pyplot as plt


In [6]:
# setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# Data Transforming as for CIFAR10 datasets
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [32]:
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

### Model

In [39]:
class CustomModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomModel, self).__init__()

        # Stage 1: 32 filters
        self.stage1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Stage 2: 64 filters
        self.stage2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Stage 3: 128 filters
        self.stage3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Global Average Pooling + Classifier
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x



In [40]:
# creating Model
model = CustomModel().to(device)

In [41]:
# train setup( loss function & )
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr = 0.0003 ,weight_decay= 0.0001 )

# Learning rate sceduler
scheduler = OneCycleLR(optimizer, max_lr= 0.001,
                       epochs= 80,
                       steps_per_epoch= len(trainloader))

# accuracy function
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [46]:
# Training function
def train(epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total
    if epoch % 10 == 0:
      print(f'Epoch: {epoch} | Loss: {running_loss/len(trainloader):.3f} | Acc: {acc:.3f}%')


In [47]:
# Test function
def test(epoch):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total
    if epoch % 10 == 0:
      print(f'Test Accuracy: {acc:.3f}%')
    return acc

In [48]:
print("Starting training...")
best_acc = 0
for epoch in range(1, 31):
    train(epoch)
    acc = test(epoch)

    if acc > best_acc:
        best_acc = acc
        print(f'New best accuracy: {best_acc:.3f}%')

print(f'Best test accuracy: {best_acc:.3f}%')

Starting training...
New best accuracy: 82.150%
New best accuracy: 82.430%
New best accuracy: 82.730%
New best accuracy: 85.270%
Epoch: 10 | Loss: 0.313 | Acc: 89.192%
Test Accuracy: 84.520%
New best accuracy: 85.940%
New best accuracy: 86.240%
New best accuracy: 86.970%
Epoch: 20 | Loss: 0.229 | Acc: 92.090%
Test Accuracy: 85.420%
New best accuracy: 87.960%
New best accuracy: 88.040%
Epoch: 30 | Loss: 0.167 | Acc: 94.160%
Test Accuracy: 87.910%
Best test accuracy: 88.040%
